In [ ]:
from DICOMLib import DICOMUtils
from DICOMLib import DICOMLoadable
import vtkSegmentationCorePython as vtkSegmentationCore
import numpy as np
import slicer
import vtk
import logging
from DICOMScalarVolumePlugin import DICOMScalarVolumePluginClass
import os
from glob import glob
import DICOMScalarVolumePlugin


# Code from https://github.com/SlicerRt/SlicerRT/blob/master/BatchProcessing/BatchStructureSetConversion.py
def ConvertStructureSetToLabelmap():
    # ConvertStructureSetToLabelmap()
    labelmapsToSave = []

    # Get all segmentation nodes from the scene
    segmentationNodes = slicer.util.getNodes('vtkMRMLSegmentationNode*')

    for segmentationNode in segmentationNodes.values():
        # Set referenced volume as rasterization reference
        referenceVolume = slicer.vtkSlicerDicomRtImportExportModuleLogic.GetReferencedVolumeByDicomForSegmentation(
            segmentationNode)
        if referenceVolume == None:
            continue

    # Perform conversion
    binaryLabelmapRepresentationName = vtkSegmentationCore.vtkSegmentationConverter.GetSegmentationBinaryLabelmapRepresentationName()
    segmentation = segmentationNode.GetSegmentation()
    segmentation.CreateRepresentation(binaryLabelmapRepresentationName)

    # Create labelmap volume nodes from binary labelmaps
    segmentIDs = vtk.vtkStringArray()
    segmentation.GetSegmentIDs(segmentIDs)
    for segmentIndex in range(0, segmentIDs.GetNumberOfValues()):
        segmentID = segmentIDs.GetValue(segmentIndex)
        segment = segmentation.GetSegment(segmentID)
        binaryLabelmap = segment.GetRepresentation(
            vtkSegmentationCore.vtkSegmentationConverter.GetSegmentationBinaryLabelmapRepresentationName())
        if not binaryLabelmap:
            continue
        labelmapNode = slicer.vtkMRMLLabelMapVolumeNode()
        slicer.mrmlScene.AddNode(labelmapNode)
        labelmapName = segmentationNode.GetName() + "_" + segmentID
        labelmapNode.SetName(labelmapName)
        if not slicer.vtkSlicerSegmentationsModuleLogic.CreateLabelmapVolumeFromOrientedImageData(
          binaryLabelmap, labelmapNode):
            continue

        # Append volume to list
        labelmapsToSave.append(labelmapNode)# SaveLabelmaps into output_dir

    return labelmapsToSave


def SaveLabelmaps(labelmapsToSave, outputDir):
    for labelmapNode in labelmapsToSave:
        # Clean up file name and set path
        fileName = labelmapNode.GetName() + '.nrrd'
        table = str.maketrans(dict.fromkeys('!?:;'))
        fileName = fileName.translate(table)
        filePath = outputDir + '/' + fileName
        logging.info('  Saving structure ' + labelmapNode.GetName() + '\n    to file ' + fileName)

    # Save to file
    success = slicer.util.saveNode(labelmapNode, filePath)
    if not success:
        logging.error('Failed to save labelmap: ' + filePath)


# Exporting data from all testing/training brachytherapy patients
folder_prefix = '/Volumes/ExFat/Data/DICOM files/PYPB'


for patient_num in range(55,101):
    # Clear scene
    slicer.mrmlScene.Clear(False)

    #concatenate folder name
    folder_name = folder_prefix + str(patient_num).zfill(2)
    
    # Load dicom folder
    DICOMUtils.openTemporaryDatabase()
    DICOMUtils.importDicom(folder_name)

    #Load first patient into slicer
    patient = slicer.dicomDatabase.patients()[0]
    DICOMUtils.loadPatientByUID(patient)

    output_dir = slicer.app.temporaryPath + '/BrachyPatients/Output/' + str(patient_num).zfill(2)
    labelmaps = ConvertStructureSetToLabelmap()
    SaveLabelmaps(labelmaps, output_dir)
    
    # Get first plan node
    planNodes = slicer.util.getNodes('vtkMRMLRTPlanNode*')
    planNode = list(planNodes.values())[0]

    with DICOMUtils.TemporaryDICOMDatabase() as db:
        result = DICOMUtils.importDicom(folder_name)
        scalarVolumeReader = DICOMScalarVolumePlugin.DICOMScalarVolumePluginClass()
        for patient in db.patients():
            for study in db.studiesForPatient(patient):
                for series in db.seriesForStudy(study):
                    print('looking at series ' + series + ' for patient ' + patient)
                    files = db.filesForSeries(series)
                    try:
                        loadable = scalarVolumeReader.examineForImport([files])[0]
                        scalarVolumeReader.load(loadable)
                    except:
                        print("error")
    
    # Get ultrasound volume node
    ultrasoundNodes = slicer.util.getNodes('vtkMRMLScalarVolumeNode*')
    ultrasoundNode = list(ultrasoundNodes.values())[-1]

    
    # Get catheter nodes (children of plan in SH)
    shNode = slicer.vtkMRMLSubjectHierarchyNode.GetSubjectHierarchyNode(slicer.mrmlScene)
    planItemID = shNode.GetItemByDataNode(planNode)
    children = vtk.vtkIdList()
    shNode.GetItemChildren(planItemID, children)

    catheterNodes = []
    for i in range(children.GetNumberOfIds()):
        child = children.GetId(i)
        catheterNode = shNode.GetItemDataNode(child)
        catheterNodes.append(catheterNode)

    # Convert each catheter node into labelmap nodes
    modelNodes = []
    for catheterNode in catheterNodes:
        # 1. Get model from catheter markups curve
        catheterNode.SetCurveTypeToLinear()
        polydata = catheterNode.GetCurveWorld()
        # add catheter diameter
        polydataTube = vtk.vtkTubeFilter()
        polydataTube.SetInputData(polydata)
        polydataTube.SetNumberOfSides(20)
        # 5F catheter radius
        polydataTube.SetRadius(0.7366)
        polydataTube.Update()
        # show on slicer for visualizing purposes
        modelNode = slicer.vtkMRMLModelNode()
        slicer.mrmlScene.AddNode(modelNode)
        modelNode.SetAndObservePolyData(polydataTube.GetOutput())
        # modelNode.CreateDefaultDisplayNodes()
        modelNodes.append(modelNode)

    #Create segmentation node
    segmentationNode = slicer.mrmlScene.AddNewNodeByClass('vtkMRMLSegmentationNode')
#     Convert all model nodes to segments
    
    for i, modelNode in enumerate(modelNodes):
        modelSegment = slicer.vtkSlicerSegmentationsModuleLogic().CreateSegmentFromModelNode(modelNode)
        segmentationNode.GetSegmentation().AddSegment(modelSegment)
        print(i)
        
    # Display segmentation node 
    slicer.mrmlScene.AddNode(segmentationNode)
    segmentationNode.CreateDefaultDisplayNodes()
    
    # Create LabelMapVolumeNode for all segments
    labelmapVolumeNode = slicer.mrmlScene.AddNewNodeByClass('vtkMRMLLabelMapVolumeNode')
    # Export multiple segments to label map volume node
    slicer.modules.segmentations.logic().ExportVisibleSegmentsToLabelmapNode(segmentationNode, labelmapVolumeNode, ultrasoundNode)
    
    # Export Segmentation to numpy array
    output_file = '/Volumes/ExFat/Data/New_Data/brachy_patient' + str(patient_num).zfill(2)
    segmentationArray = slicer.util.arrayFromVolume(labelmapVolumeNode)
    np.save(output_file, segmentationArray)

    # Export ultrasound volume node to numpy array
    ultrasound_file = '/Volumes/ExFat/Data/New_Data/ultrasound' + str(patient_num).zfill(2)
    ultrasoundArray = slicer.util.arrayFromVolume(ultrasoundNode)
    np.save(ultrasound_file, ultrasoundArray)


Switching to temporary DICOM database: /private/var/folders/p3/txcp4byn3nxdz_jm8p86m6gr0000gn/T/Slicer-nikki/20221220_195421_TempDICOMDatabase
  Saving structure 1: RTSTRUCT: OCP RTS v4.2.2._CTV1
    to file 1 RTSTRUCT OCP RTS v4.2.2._CTV1.nrrd
  Saving structure 1: RTSTRUCT: OCP RTS v4.2.2._CTV2
    to file 1 RTSTRUCT OCP RTS v4.2.2._CTV2.nrrd
  Saving structure 1: RTSTRUCT: OCP RTS v4.2.2._Urethra
    to file 1 RTSTRUCT OCP RTS v4.2.2._Urethra.nrrd
  Saving structure 1: RTSTRUCT: OCP RTS v4.2.2._Rectum
    to file 1 RTSTRUCT OCP RTS v4.2.2._Rectum.nrrd
Switching to temporary DICOM database: /private/var/folders/p3/txcp4byn3nxdz_jm8p86m6gr0000gn/T/Slicer-nikki/20221220_195432_TempDICOMDatabase
looking at series 1.3.6.1.4.1.5962.99.1.3773856834.1244485066.1635861462082.366.0 for patient 1
Loading with imageIOName: GDCM


Cannot get DICOM slice positions for volume 1: Unnamed Series


looking at series 1.3.6.1.4.1.5962.99.1.3773856834.1244485066.1635861462082.369.0 for patient 1
Loading with imageIOName: GDCM


Could not read scalar volume using GDCM approach.  Error is: FileFormatError


Loading with imageIOName: DCMTK


Could not read scalar volume using DCMTK approach.  Error is: FileFormatError


looking at series 1.3.6.1.4.1.5962.99.1.3773856834.1244485066.1635861462082.437.0 for patient 1
error
looking at series 1.3.6.1.4.1.5962.99.1.3773856834.1244485066.1635861462082.370.0 for patient 1
Loading with imageIOName: GDCM
0
1
2
3
4
5
